In [21]:
import submitit
from frust.stepper import Stepper
from pathlib import Path
from tooltoad.chemutils import xyz2mol

# Run single structure

In [22]:
f = Path("../structures/ts4_TMP.xyz")
mols = {}
with open(f, "r") as file:
    xyz_block = file.read()
    mol = xyz2mol(xyz_block)
    mols[f.stem] = (mol, [0])

step = Stepper(list(mols.keys()), step_type="ts1", save_output_dir=False)
df0 = step.build_initial_df(mols)

2025-08-31 08:37:04 INFO  frust.stepper: Working dir: .


In [ ]:
def run_orca_calc(
    df,
    results_dir,
    debug=False,
    n_cores=10,
    mem_gb=30,
):
    from pathlib import Path
    import nuse
    nuse.start_monitoring(filter_cgroup=True)

    name = df["custom_name"].iloc[0]

    results_dir = Path(results_dir)
    results_dir.mkdir(exist_ok=True)

    step = Stepper([name],
                step_type="none",
                debug=debug,
                save_output_dir=True,
                save_calc_dirs=True,
                output_base=str(results_dir),
                n_cores=n_cores,
                memory_gb=mem_gb)

    df1 = step.orca(df, "UMA-test-1", {"OptTS": None, "ExtOpt": None}, uma="omol", xtra_inp_str=
"""%geom
  Calc_Hess  true
  NumHess    true
  Recalc_Hess 5
  MaxIter    300
end""")

    df1.to_parquet(f"{results_dir}/results_{name}.parquet")

In [24]:
DEBUG           = False
N_CORES         = 20
MEM_GB          = 40
TIMEOUT_MIN     = 14400
RESULTS_DIR     = "UMA-test"

executor = submitit.AutoExecutor(f"logs/{RESULTS_DIR}")
executor.update_parameters(
    slurm_partition="kemi1",
    cpus_per_task=N_CORES,
    mem_gb=MEM_GB,
    timeout_min=TIMEOUT_MIN,
    slurm_additional_parameters={"exclude": "node070"},
)

name = df0["custom_name"].iloc[0]
executor.update_parameters(slurm_job_name=name)
executor.submit(run_orca_calc, df0, RESULTS_DIR, DEBUG, N_CORES, MEM_GB)
print(f"Submitted: {name}")

Submitted: ts4_TMP
